In [1]:
from pathlib import Path
import pandas as pd
import shutil
import yaml

from utils import (
    parse_ann_files,
    save_json,
)

DATASETS_DIR = Path("./datasets")

EHR

In [14]:
# EHR
ehr_dir = DATASETS_DIR / 'ehr'
diagnosis2icd_train = parse_ann_files(ehr_dir)
# test_dir = DATASETS_DIR / 'test'
# diagnosis2icd_test = parse_ann_files(test_dir)
# diagnosis2icd = dict(list(diagnosis2icd_train.items()) + list(diagnosis2icd_test.items()))
# save_json(diagnosis2icd, DATASETS_DIR /'ehr-diagnosis2icd.json')
save_json(diagnosis2icd_train, DATASETS_DIR / 'ehr-train-diagnosis2icd.json')
len(diagnosis2icd_train)

4007

MKB

In [13]:
# MKB full
mkb_icd2diagnosis = yaml.safe_load(Path(DATASETS_DIR / 'mkb2descr.yaml').read_text())
print(len(mkb_icd2diagnosis))
mkb_full_diagnosis2icd = {}
for k, v in mkb_icd2diagnosis.items():
    v = v.lower()
    # Save first occurence (group code, not specific)
    if v not in mkb_full_diagnosis2icd:
        mkb_full_diagnosis2icd[v] = k
save_json(mkb_full_diagnosis2icd, DATASETS_DIR /'mkb-full-diagnosis2icd.json')
len(mkb_full_diagnosis2icd)

17762


17355

In [ ]:
# duplicated = ddf[ddf['diagnosis'].duplicated()]['diagnosis'].to_list()

# def combined_diagnosis(row):
#     if row.diagnosis in duplicated:
#         group_diagnosis = ddf[ddf.index == row.name.split('.')[0]]['diagnosis'].to_string(index=False)
#         duplicated.remove(row.diagnosis)
#         return f"{row.diagnosis} ({group_diagnosis})"
#     return row.diagnosis

# ddf['diagnosis'] = ddf.apply(combined_diagnosis, axis=1)

In [107]:
# MKB
mkb_file_path = DATASETS_DIR / 'MKB10_202405280919.xlsx'
df = pd.read_excel(mkb_file_path)
df.columns = ['icd', 'diagnosis']
df.drop(2010, inplace=True)  # Drop header
# df.drop_duplicates(inplace=True)
raw_df = df.copy()

# df.dropna(inplace=True)
nan_df = df[df['icd'].isna() & df['diagnosis'].notna()]
nan_df['icd'] = nan_df['diagnosis'].str.extract(r'\(([^)]+)\)$', expand=False).fillna(nan_df['icd']).astype(str).apply(lambda s: s.strip())
nan_df['diagnosis'] = nan_df['diagnosis'].str.replace(r'\s*\(.*?\)', '', regex=True)
df.loc[nan_df.index, :] = nan_df
df.drop(index=df[df['icd'] == 'nan'].index, inplace=True)

# df.set_index('icd', inplace=True)
# mkb_icd2diagnosis = df.to_dict()['diagnosis']
# mkb_diagnosis2icd = {v.lower(): k for k, v in mkb_icd2diagnosis.items()}
# save_json(mkb_diagnosis2icd, DATASETS_DIR /'mkb-r-diagnosis2icd.json')
# len(mkb_diagnosis2icd)

/tmp/ipykernel_1873281/780299714.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_df['icd'] = nan_df['diagnosis'].str.extract(r'\(([^)]+)\)$', expand=False).fillna(nan_df['icd']).astype(str).apply(lambda s: s.strip())
/tmp/ipykernel_1873281/780299714.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nan_df['diagnosis'] = nan_df['diagnosis'].str.replace(r'\s*\(.*?\)', '', regex=True)


In [15]:
mkb_full_ehr_train_diagnosis2icd = dict(list(mkb_full_diagnosis2icd.items()) + list(diagnosis2icd_train.items()))
save_json(mkb_full_ehr_train_diagnosis2icd, DATASETS_DIR /'mkb-full-ehr-train-diagnosis2icd.json')
len(mkb_full_ehr_train_diagnosis2icd)

21197

RUCCON

In [ ]:
# RUCCON
raw_ruccon_dir = Path('../RuCCoN/RUCCON')
ruccon_dir = Path(DATASETS_DIR / 'ruccon')
ruccon_dir.mkdir(parents=True, exist_ok=True)
index = 0

for main_dir in raw_ruccon_dir.iterdir():
    if main_dir.is_dir() and main_dir.name.startswith("main"):
        # Check for subdirectories within /main{N}/*
        for sub_dir in main_dir.iterdir():
            if sub_dir.is_dir():
                ann_files = sorted(sub_dir.glob("*.ann"))
                txt_files = sorted(sub_dir.glob("*.txt"))
                for file_ann, file_txt in zip(ann_files, txt_files):
                    shutil.copy(file_ann, ruccon_dir / f'{index}_{file_ann.stem}{file_ann.suffix}')
                    shutil.copy(file_txt, ruccon_dir / f'{index}_{file_txt.stem}{file_txt.suffix}')
                    index += 1
        # Also check for files directly in /main{N}/
        ann_files = sorted(main_dir.glob("*.ann"))
        txt_files = sorted(main_dir.glob("*.txt"))
        for file_ann, file_txt in zip(ann_files, txt_files):
            shutil.copy(file_ann, ruccon_dir / f'{index}_{file_ann.stem}{file_ann.suffix}')
            shutil.copy(file_txt, ruccon_dir / f'{index}_{file_txt.stem}{file_txt.suffix}')
            index += 1

In [74]:
umls_to_icd_file = DATASETS_DIR / "cui2icds_ICD10_ICD10CM.txt"
umls_to_icd = {}

with umls_to_icd_file.open("r", encoding="utf-8") as file:
    for line in file:
        umls_code, icd_codes = line.strip().split("\t")
        # Find the shortest ICD code, prioritize those with '-'
        icd_code = sorted(icd_codes.split('|'), key=lambda x: ('-' not in x, len(x)))[0]
        umls_to_icd[umls_code] = icd_code

In [95]:
ruccon_dir = DATASETS_DIR / 'ruccon'
ruccon_diagnosis2icd = {}
for file_ann in ruccon_dir.glob("*.ann"):
    ann_text = file_ann.read_text(encoding="utf-8")
    t_numbers = []
    references = {}
    for line in ann_text.splitlines():
        if line.startswith("T"):
            t_number = line.split("\t")[0]
            t_numbers.append(t_number)
        elif line.startswith("N"):
            n_number, reference, entity_string = line.split("\t", 2)
            _, t_ref, umls = reference.split(' ', 2)
            if t_ref in t_numbers:
                _, umls_code = umls.split(":")
                references[t_ref] = (entity_string, umls_code)
    for t_number in t_numbers:
        diagnosis_umls = references.get(t_number, None)
        if diagnosis_umls:
            diagnosis, umls_code = diagnosis_umls
            diagnosis = diagnosis.strip().lower()
            icd_code = umls_to_icd.get(umls_code, None)
            if icd_code:
                ruccon_diagnosis2icd[diagnosis] = icd_code

In [96]:
len(ruccon_diagnosis2icd)

627

In [60]:
save_json(ruccon_diagnosis2icd, DATASETS_DIR /'ruccon-diagnosis2icd.json')

NEREL-BIO

In [10]:
# https://github.com/nerel-ds/NEREL-BIO/tree/master/nerel-bio-v1.0/train
# NEREL-BIO
raw_nerel_bio_dir = Path('../NEREL-BIO/nerel-bio-v1.0/train')
nerel_bio_dir = Path(DATASETS_DIR / 'nerel-bio')
nerel_bio_dir.mkdir(parents=True, exist_ok=True)

for file_ann, file_txt in zip(raw_nerel_bio_dir.glob("*.ann"), raw_nerel_bio_dir.glob("*.txt")):
    shutil.copy(file_ann, nerel_bio_dir / file_ann.name)
    shutil.copy(file_txt, nerel_bio_dir / file_txt.name)

In [87]:
umls_to_icd_file = DATASETS_DIR / "cui2icds_ICD10_ICD10CM.txt"
umls_to_icd = {}

with umls_to_icd_file.open("r", encoding="utf-8") as file:
    for line in file:
        umls_code, icd_codes = line.strip().split("\t")
        # Find the shortest ICD code, prioritize those with '-'
        icd_code = sorted(icd_codes.split('|'), key=lambda x: ('-' not in x, len(x)))[0]
        umls_to_icd[umls_code] = icd_code

In [91]:
vocab_path = DATASETS_DIR / "vocab_umls_rus_biosyn.txt"
diagnosis2umls = {}
with vocab_path.open("r", encoding="utf-8") as file:
    for line in file:
        umls_code, diagnosis = line.strip().split("||")
        diagnosis = diagnosis.strip().lower()
        if diagnosis not in diagnosis2umls:
            diagnosis2umls[diagnosis] = umls_code

In [92]:
nerel_bio_path = DATASETS_DIR / 'nerel-bio'
nerel_bio_diagnosis2icd = {}
for file_ann in nerel_bio_path.glob("*.ann"):
    ann_text = file_ann.read_text(encoding="utf-8")
    for line in ann_text.splitlines():
        if line.startswith("T"):
            _, _, extracted_string = line.split("\t", 2)
            extracted_string = extracted_string.strip().lower()
            umls_code = diagnosis2umls.get(extracted_string, None)
            if umls_code:
                icd_code = umls_to_icd.get(umls_code, None)
                if icd_code:
                    nerel_bio_diagnosis2icd[extracted_string] = icd_code

In [93]:
len(nerel_bio_diagnosis2icd)

264

In [94]:
save_json(nerel_bio_diagnosis2icd, DATASETS_DIR /'nerel-bio-diagnosis2icd.json')